<font size="+3">Part IV - Decompounded Options</font>

| Name             | Student ID |
|------------------|------------|
| Ishani Maitra| [01506460]  |
| Nandini Agarwal    | [01518145]|

>[Models](#scrollTo=0gz-RcZKKxX0)

>[CMS Payoff](#scrollTo=supOxc1kI57N)

>[Data Preparation](#scrollTo=UfM6BYM5rb3m)

>[Question 1](#scrollTo=4jR-ivhWyr46)

>[Question 2](#scrollTo=kGMVliX7yuE-)



In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.stats import norm
from scipy import interpolate
from math import log, sqrt, exp
from scipy import integrate
import warnings
warnings.filterwarnings("ignore")

#Models

In [103]:
def black76_call(F, K, sigma, T):
    """
    Price a European call option using the Black-76 formula.

    Parameters:
        F (float): Forward rate.
        K (float): Strike rate.
        PVBP (float): Present Value of a Basis Point (annuity factor).
        sigma (float): Volatility.
        T (float): Time to maturity (in years).

    Returns:
        float: Call option price.
    """
    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return (F * norm.cdf(d1) - K * norm.cdf(d2))

def black76_put(F, K, sigma, T):
    """
    Price a European put option using the Black-76 formula.

    Parameters:
        F (float): Forward rate.
        K (float): Strike rate.
        PVBP (float): Present Value of a Basis Point (annuity factor).
        sigma (float): Volatility.
        T (float): Time to maturity (in years).

    Returns:
        float: Put option price.
    """
    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return (K * norm.cdf(-d2) - F * norm.cdf(-d1))

In [104]:
def sabr_implied_vol(F, K, T, alpha, beta, rho, nu):
    """
    Compute the SABR model implied volatility given market inputs.

    Parameters:
        F (float): Forward rate.
        K (float): Strike rate.
        T (float): Time to maturity (years).
        alpha (float): SABR alpha parameter.
        beta (float): SABR beta parameter.
        rho (float): SABR rho parameter.
        nu (float): SABR nu parameter (volatility of volatility).

    Returns:
        float: Implied volatility computed by the SABR model.
    """
    # Use K directly as X
    X = K

    # At-the-money (ATM) case
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta) ** 2) / 24) * (alpha ** 2) / (F ** (2 - 2 * beta))
        numer2 = 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
        numer3 = ((2 - 3 * rho ** 2) / 24) * (nu ** 2)
        VolAtm = alpha * (1 + (numer1 + numer2 + numer3) * T) / (F ** (1 - beta))
        sabr_sigma = VolAtm
    else:
        # Non-ATM case: compute the intermediate variables
        z = (nu / alpha) * ((F * X) ** (0.5 * (1 - beta))) * np.log(F / X)
        zhi = np.log((np.sqrt(1 - 2 * rho * z + z ** 2) + z - rho) / (1 - rho))
        numer1 = (((1 - beta) ** 2) / 24) * (alpha ** 2) / ((F * X) ** (1 - beta))
        numer2 = 0.25 * rho * beta * nu * alpha / ((F * X) ** ((1 - beta) / 2))
        numer3 = ((2 - 3 * rho ** 2) / 24) * (nu ** 2)
        numer = alpha * (1 + (numer1 + numer2 + numer3) * T) * z

        denom1 = ((1 - beta) ** 2 / 24) * (np.log(F / X)) ** 2
        denom2 = (((1 - beta) ** 4) / 1920) * (np.log(F / X)) ** 4
        denom = ((F * X) ** ((1 - beta) / 2)) * (1 + denom1 + denom2) * zhi

        sabr_sigma = numer / denom

    return sabr_sigma

#CMS Payoff

For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

  \begin{equation*}
    \begin{split}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\mbox{IRR}(K)} \\
      h'(K) &= \frac{\mbox{IRR}(K)g'(K) - g(K)\mbox{IRR}'(K)}{\mbox{IRR}(K)^2} \\
      h''(K) &= \frac{\mbox{IRR}(K)g''(K)-\mbox{IRR}''(K)g(K) -2\cdot\mbox{IRR}'(K)g'(K)}{\mbox{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\mbox{IRR}'(K)^2g(K)}{\mbox{IRR}(K)^3}.
    \end{split}
  \end{equation*}
  
For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

We can implement this in Python. First we define the IRR functions.

Let $m$ denote the payment frequenc ($m=2$ for semi-annual payment frequency), and let $N = T_N-T_n$ denote the tenor of the swap (number of years), the partial derivatives on the IRR function $\mbox{IRR}(S)$ given by:
\begin{equation*}
\begin{split}
\mbox{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\mbox{IRR}'(K)&=-\frac{1}{K}\mbox{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\mbox{IRR}''(K)&=-\frac{2}{K}\mbox{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

These results will need to be generalised to handle the case for $m=2$ to be consistent with the semi-annual payment frequency swap market data provided.


In [105]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative


For CMS rate payment, since $g(F) = F^{\frac{1}{4}} - 0.04^{\frac{1}{4}}$, $p=4$, $q=2$, we have the derivatives:

$$
g(K) = K^{\frac{1}{4}} - 0.04^{\frac{1}{4}}
$$

$$
g'(K) = \frac{1}{4} K^{-\frac{3}{4}}
$$

$$
g''(K) = -\frac{3}{16} K^{-\frac{7}{4}}
$$

In [106]:

p = 4
q = 2
const_val = 0.04 #from diff
# assembling them
def g_0(K, p=p, q=q, c=const_val):
    return K**(1/p) - c**(1/q)

def g_1(K, p=p):
    return (1/p) * K**((1/p) - 1)

def g_2(K, p=p):
    return (1/p) * ((1/p) - 1) * K**((1/p) - 2)

The function $h(K) = g(K)/IRR(K)$ now simplies into:  
  
  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\mbox{IRR}(K)} \\
      h'(K) &= \frac{\mbox{IRR}(K)g'(K) - g(K)\mbox{IRR}'(K)}{\mbox{IRR}(K)^2} \\
      h''(K) &= \frac{\mbox{IRR}(K)g''(K)-\mbox{IRR}''(K)g(K) -2\cdot\mbox{IRR}'(K)g'(K)}{\mbox{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\mbox{IRR}'(K)^2g(K)}{\mbox{IRR}(K)^3}.
    \end{split}
  \end{equation*}





In [107]:
def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

We will also need to implement the IRR-settled payer and receiver swaption formulae:

  \begin{equation*}
    \begin{split}
      V^{pay}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Call}(S_{n,N}(0),K,\sigma_{n,N},T) \\
      V^{rec}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Put}(S_{n,N}(0),K,\sigma_{n,N},T) \\
    \end{split}
  \end{equation*}

where $S_{n,N}(0)=F$ is today's forward swap rate calculated based on the curves we bootstrapped, and $\sigma_{n,N}$ is the SABR implied volatility calibrated to swaption market data.

In [108]:
def swaption(discount, F, K, sigma, T, m, N, valuationtype):
    if valuationtype.lower() == 'payer':
        return discount * IRR_0(F, m, N) * black76_call(F, K, sigma, T)
    elif valuationtype.lower() == 'receiver':
        return discount * IRR_0(F, m, N) * black76_put(F, K, sigma, T)
    else:
      print("error")

def swaption_integrand(discount, F, K, sigma, T, m, N, payofftype):
    if payofftype.lower() == 'call':
        return h_2(K, m, N) * swaption(discount, F, K, sigma, T, m, N, 'payer')
    elif payofftype.lower() == 'put':
        return h_2(K, m, N) * swaption(discount, F, K, sigma, T, m, N, 'receiver')

#Data Preparation

In [109]:
calibrated_params_df = pd.read_csv("calibrated_params.csv")
discount_df = pd.read_csv("Discount_Factor.csv")
swapR_df = pd.read_csv("Forward_Swap_Rate.csv")

In [110]:
swapR_df.set_index('Unnamed: 0', inplace=True)
swapR_df = swapR_df.T.unstack().reset_index(name='Swap_Rate')
swapR_df.columns = ["Expiry", "Tenor", "Swap_Rate"]
swapR_df

,Expiry,Tenor,Swap_Rate
0,1Y,1Y,0.032007
1,1Y,2Y,0.033259
2,1Y,3Y,0.034011
3,1Y,5Y,0.035255
4,1Y,10Y,0.038428
5,5Y,1Y,0.039274
6,5Y,2Y,0.040075
7,5Y,3Y,0.040072
8,5Y,5Y,0.041093
9,5Y,10Y,0.043634


#Question 1

In [111]:
# All parameters put here, only focus T=5y CMS10y
T, N, m, beta = 5, 10, 2, 0.9 # note m=2, semiannual basis from ir excel data
discount = discount_df[(discount_df.Tenor == T)]["OIS Discount Factor"].iloc[0]
F = swapR_df[(swapR_df.Expiry == '5Y') & (swapR_df.Tenor == '10Y')]['Swap_Rate'].iloc[0]

alpha = calibrated_params_df[(calibrated_params_df.Expiry == '5Y') & (calibrated_params_df.Tenor == '10Y')]['Alpha'].iloc[0]
rho = calibrated_params_df[(calibrated_params_df.Expiry == '5Y') & (calibrated_params_df.Tenor == '10Y')]['Rho'].iloc[0]
nu=calibrated_params_df[(calibrated_params_df.Expiry == '5Y') & (calibrated_params_df.Tenor == '10Y')]['Nu'].iloc[0]


In [112]:
print ('discount:', discount)
print ('F:', F)
print ('alpha:', alpha)
print ('rho:', rho)
print ('nu:', nu)
print ('beta', beta)
print ('T, N, m = ', T,',', N,',', m)


discount: 0.982184119733244
F: 0.0436336455274748
alpha: 0.1699086246926807
rho: -0.3651234699171674
nu: 0.5396558283943024
beta 0.9
T, N, m =  5 , 10 , 2


In [113]:
CMS_put = quad(lambda x: swaption_integrand(discount,
                                          F,
                                          x,
                                          sabr_implied_vol(F,x,T, alpha=alpha, beta=beta, rho=rho, nu=nu),
                                          T,
                                          m ,
                                          N,
                                          'put'),
              0.0, F, limit=200)

CMS_call = quad(lambda x: swaption_integrand(discount,
                                             F,
                                             x,
                                             sabr_implied_vol(F,x,T, alpha=alpha, beta=beta, rho=rho, nu=nu),
                                             T,
                                             m ,
                                             N,
                                             'call'),
                 F, 1000, limit=200)

decompounded_payoff = discount * g_0(F) + CMS_put[0] + CMS_call[0]

print(f"The PV of the payoff of the decompounded option in part 1 is: {decompounded_payoff}")

The PV of the payoff of the decompounded option in part 1 is: 0.23222043883303178


#Question 2

**The payoff is now:**

$ (\text{CMS 10y}^{\tfrac{1}{p}} - 0.04^{\tfrac{1}{q}} )^{+} $

Use static replication to value the PV of this payoff.


$$
V_0 = h'(L)V^{pay}(L) + \int_{L}^{\infty} h''(K)V^{pay}(K)dK
$$

For CMS rate payoff, the payoff function can be defined simply as $g(F) = F^{\frac{1}{p}} - 0.04^{\frac{1}{q}}$.

$$
F^{\frac{1}{4}} > 0.2
$$

$$
F > 0.2^4 = L
$$


In [114]:
# Define the threshold strike K for the positive part (with p=4 and q=2)
thresholdK = (0.04**(1/2))**4  # equivalent to (sqrt(0.04))^4

# Compute the call-side integrand integral from L to 5000 using swaption_integrand (for payer swaption)
I_call2, err_call2 = quad(lambda x: swaption_integrand(discount,

                                                       F,
                                                       x,
                                                       sabr_implied_vol(F, x, T, alpha=alpha, beta=beta, rho=rho, nu=nu),
                                                       T,
                                                       m,
                                                       N,
                                                       'call'
                                                       ),
                          thresholdK,5000)

# Combine the upfront term with the integrated part.
# h_1(L, m, N) is the first derivative of h evaluated at L,
# and pay_swaption gives the payer swaption price at strike L.
p2 = h_1(thresholdK, m, N) * swaption(discount,
                                      F,
                                      thresholdK,
                                      sabr_implied_vol(F, thresholdK, T, alpha=alpha, beta=beta, rho=rho, nu=nu),
                                      T,
                                      m,
                                      N,
                                      'payer'
                                      )
p2 = p2 + I_call2

print(f"PV of part 2 payoff is: {p2}")

PV of part 2 payoff is: 0.2539295053017896
